<a href="https://colab.research.google.com/github/navin0077/Face-Analysis-Tool/blob/main/face_analysis_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python-headless
!pip install dlib
!pip install scipy


In [5]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance
import urllib.request

url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
urllib.request.urlretrieve(url, "shape_predictor_68_face_landmarks.dat.bz2")

import bz2, shutil

with bz2.BZ2File("shape_predictor_68_face_landmarks.dat.bz2") as fr, open("shape_predictor_68_face_landmarks.dat", "wb") as fw:
    shutil.copyfileobj(fr, fw)

# Load face detector & landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def get_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    if len(faces) == 0:
        raise Exception("No face detected! Please upload a clear frontal face image.")

    shape = predictor(gray, faces[0])
    landmarks = np.array([[p.x, p.y] for p in shape.parts()])
    return landmarks

def calculate_symmetry(landmarks):
    left_points = landmarks[0:8]
    right_points = landmarks[9:17]
    right_points = np.flip(right_points, axis=0)
    symmetry_score = np.mean([distance.euclidean(l, r) for l, r in zip(left_points, right_points)])
    return symmetry_score

def detect_face_shape(landmarks):
    width = distance.euclidean(landmarks[0], landmarks[16])
    height = distance.euclidean(landmarks[8], landmarks[27])
    jaw_width = distance.euclidean(landmarks[4], landmarks[12])

    ratio = width / height

    if ratio < 0.85:
        return "Oval"
    elif ratio > 1.05:
        return "Round"
    elif jaw_width / width < 0.8:
        return "Heart"
    else:
        return "Square"

def analyze_jawline(landmarks):
    jaw_angle = np.degrees(np.arctan2(landmarks[8][1] - landmarks[4][1], landmarks[8][0] - landmarks[4][0]))
    if jaw_angle > 45:
        return "Sharp Jawline"
    else:
        return "Soft Jawline"

def detect_skin_tone(image, landmarks):
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    points = cv2.convexHull(landmarks)
    cv2.fillConvexPoly(mask, points, 255)

    face_pixels = cv2.bitwise_and(image, image, mask=mask)
    b, g, r = cv2.mean(face_pixels, mask=mask)[:3]

    if r > 200 and g > 180:
        return "Fair"
    elif r > 150 and g > 120:
        return "Medium"
    else:
        return "Dark"

def analyze_face(image_path):
    image = cv2.imread(image_path)
    landmarks = get_landmarks(image)

    face_shape = detect_face_shape(landmarks)
    symmetry = calculate_symmetry(landmarks)
    jawline = analyze_jawline(landmarks)
    skin_tone = detect_skin_tone(image, landmarks)

    return {
        "Face Shape": face_shape,
        "Symmetry Score": round(symmetry, 2),
        "Jawline": jawline,
        "Skin Tone": skin_tone
    }
from google.colab import files

uploaded = files.upload()
image_path = list(uploaded.keys())[0]

result = analyze_face(image_path)

print("🔍 Face Analysis Result:")
for key, value in result.items():
    print(f"{key}: {value}")


Saving IMG_1944.JPG to IMG_1944.JPG
🔍 Face Analysis Result:
Face Shape: Round
Symmetry Score: 438.26
Jawline: Sharp Jawline
Skin Tone: Medium
